# 这里包含了两类计算：分类和回归

In [11]:
#引入需要的python库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
import pickle
from sklearn.svm import SVR
import pymatgen as mg

# 例一：分类计算

In [24]:
#read data 读取数据
DE_c = pd.read_excel('Training_Set.xlsx',sheet_name=0)
array_c = DE_c.values   #数据数值提取
X_c = array_c[:,3:139]  #第3—139列数据作为输入变量X
Y_c = array_c[:,2]      #第2列作为目标数据Y
Y_c = Y_c.astype('int') #强制将目标数据的类型转换为整型int

In [25]:
#train/test split
X_train_c, X_test_c, Y_train_c, Y_test_c = train_test_split(X_c, Y_c, test_size=0.1, random_state=15, shuffle=True)


# train_test_split函数的简单说明
#在机器学习中，我们通常将原始数据按照比例分割为“测试集”和“训练集”，从 sklearn.model_selection 中调用train_test_split 函数 

#X_train,X_test, y_train, y_test =s klearn.model_selection.train_test_split(train_data,train_target,test_size=0.4, random_state=0,stratify=y_train)

# train_data：所要划分的样本特征集

# train_target：所要划分的样本结果

# test_size：样本占比，如果是整数的话就是样本的数量

# random_state：是随机数的种子。
# 随机数种子：其实就是该组随机数的编号，在需要重复试验的时候，保证得到一组一样的随机数。比如你每次都填1，其他参数一样的情况下你得到的随机数组是一样的。但填0或不填，每次都会不一样。

#stratify是为了保持split前类的分布。
#比如有100个数据，80个属于A类，20个属于B类。如果train_test_split(... test_size=0.25, stratify = y_all), 那么split之后数据如下： 
#training: 75个数据，其中60个属于A类，15个属于B类。 
#testing: 25个数据，其中20个属于A类，5个属于B类。 

#用了stratify参数，training集和testing集的类的比例是 A：B= 4：1，等同于split前的比例（80：20）。通常在这种类分布不平衡的情况下会用到stratify。

#将stratify=X就是按照X中的比例分配 

#将stratify=y就是按照y中的比例分配 

#整体总结起来各个参数的设置及其类型如下：

#主要参数说明：

#*arrays：可以是列表、numpy数组、scipy稀疏矩阵或pandas的数据框

#test_size：可以为浮点、整数或None，默认为None

#①若为浮点时，表示测试集占总样本的百分比

#②若为整数时，表示测试样本样本数

#③若为None时，test size自动设置成0.25

#train_size：可以为浮点、整数或None，默认为None

#①若为浮点时，表示训练集占总样本的百分比

#②若为整数时，表示训练样本的样本数

#③若为None时，train_size自动被设置成0.75

#random_state：可以为整数、RandomState实例或None，默认为None

#①若为None时，每次生成的数据都是随机，可能不一样

#②若为整数时，每次生成的数据都相同

#stratify：可以为类似数组或None

#①若为None时，划分出来的测试集或训练集中，其类标签的比例也是随机的

#②若不为None时，划分出来的测试集或训练集中，其类标签的比例同输入的数组中类标签的比例相同，可以用于处理不均衡的数据集

In [26]:
#preprocessing  预处理
scaler_c = preprocessing.StandardScaler().fit(X_train_c)

X_train_c = scaler_c.transform(X_train_c)
#print(X_train_c)
X_test_c = scaler_c.transform(X_test_c)
#print(X_test_c)

In [40]:
#model construction
classification=SVC(kernel='rbf',C=10**1.5, gamma= 0.01).fit(X_train_c, Y_train_c)
#说明：

#save model
classification_model=pickle.dumps(classification)

#pickle.dump(obj, file, protocol) 
#注释： 
#obj——序列化对象，将对象obj保存到文件file中去； 
#file——file表示保存到的类文件对象，file必须有write()接口，file可以是一个以’w’打开的文件或者是一个StringIO对象，也可以是任何可以实现write()接口的对象；
#protocol——序列化模式，默认是 0（ASCII协议，表示以文本的形式进行序列化），protocol的值还可以是1和2（1和2表示以二进制的形式进行序列化。其中，1是老式的二进制协议；2是新二进制协议）。






# 例二：回归计算

In [36]:
#read data
DE_r = pd.read_excel('Training_Set.xlsx',sheet_name=1)
array_r = DE_r.values
X_r = array_r[:,2:138]
Y_r = array_r[:,1]

In [37]:
#train/test split
X_train_r, X_test_r, Y_train_r, Y_test_r = train_test_split(X_r, Y_r, test_size=0.1, random_state=15, shuffle=True)
#preprocessing
scaler_r = preprocessing.StandardScaler().fit(X_train_r)
X_train_r = scaler_r.transform(X_train_r)
X_test_r = scaler_r.transform(X_test_r)

In [38]:
#model construction
regression = SVR(kernel='rbf',C=10, epsilon=0.1, gamma= 0.01).fit(X_train_r, Y_train_r)
predicted_Y1_r = regression.predict(X_train_r)
predicted_Y_r = regression.predict(X_test_r)
#save model
regression_model=pickle.dumps(regression)

In [45]:
prediction = pd.read_excel('to_predict.xlsx')
prediction.head()
prediction.dtypes

Composition    object
dtype: object

# 描述符的计算

In [32]:
class Vectorize_Formula:
	def __init__(self):
		elem_dict = pd.read_excel(r'elements.xlsx')
		self.element_df = pd.DataFrame(elem_dict) 
		self.element_df.set_index('Symbol',inplace=True)
		self.column_names = []
		for string in ['avg','diff','max','min']:
			for column_name in list(self.element_df.columns.values):
				self.column_names.append(string+'_'+column_name)

	def get_features(self, formula):
		try:
			fractional_composition = mg.Composition(formula).fractional_composition.as_dict()
#			element_composition = mg.Composition(formula).element_composition.as_dict()
			avg_feature = np.zeros(len(self.element_df.iloc[0]))
#			sum_feature = np.zeros(len(self.element_df.iloc[0]))
			for key in fractional_composition:
				try:
					avg_feature += self.element_df.loc[key].values * fractional_composition[key]                  
					diff_feature = self.element_df.loc[list(fractional_composition.keys())].max()-self.element_df.loc[list(fractional_composition.keys())].min()
				except Exception as e: 
					print('The element:', key, 'from formula', formula,'is not currently supported in our database')
					return np.array([np.nan]*len(self.element_df.iloc[0])*4)
			max_feature = self.element_df.loc[list(fractional_composition.keys())].max()
			min_feature = self.element_df.loc[list(fractional_composition.keys())].min()

			features = pd.DataFrame(np.concatenate([avg_feature, diff_feature, np.array(max_feature), np.array(min_feature)]))
			features = np.concatenate([avg_feature, diff_feature, np.array(max_feature), np.array(min_feature)])
			return features.transpose()
		except:
			print('There was an error with the Formula: '+ formula + ', this is a general exception with an unkown error')
			return [np.nan]*len(self.element_df.iloc[0])*4

In [33]:
gf=Vectorize_Formula()
features=[]
targets=[]

In [34]:
for formula in prediction['Composition']:
    features.append(gf.get_features(formula))
X = pd.DataFrame(features, columns = gf.column_names)
pd.set_option('display.max_columns', None)

X_c = scaler_c.transform(X)
pred_c = pickle.loads(classification_model)
result_c = pred_c.predict(X_c)
X_r = scaler_r.transform(X)
pred_r = pickle.loads(regression_model)
result_r = pred_r.predict(X_r)
result=[]
for i in range(len(result_c)):
    if result_c[i] == 1:
        result.append(result_r[i]);
    else:
        result.append(result_c[i])
result = np.around(result,decimals=2)

In [51]:
composition=pd.read_excel('to_predict.xlsx',sheet_name='Sheet1', usecols="A")
#print(composition)
composition=pd.DataFrame(composition)
#print(composition)
result=pd.DataFrame(result)
print(result)
predicted=np.column_stack((composition,result))
predicted=pd.DataFrame(predicted)
predicted.to_excel('predicted.xlsx', index=False, header=("Composition","Predicted Eg"))
print("A file named predicted.xlsx has been generated.\nPlease check your folder.")

      0
0  5.21
1  3.57
2  0.00
3  0.00
4  0.00
5  5.73
6  0.74
7  1.48
8  0.38
A file named predicted.xlsx has been generated.
Please check your folder.
